# Compare the idiom paraphrase and EN-DE idiom translation datasets

## Load Data

In [ ]:
import pandas as pd
from google.colab import drive

from random import randrange, choice
import re
import nltk
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
drive.mount('/content/drive')
dir = "/content/drive/MyDrive/MRP Idiom Translation/"

Mounted at /content/drive


In [ ]:
# # Load EN-DE dataset
# base_path=dir+"IdiomTranslationDS-master/en-de/"
# en = pd.read_csv(base_path+'idiom_test.en',sep='\n', header=None,names=['en'])
# de = pd.read_csv(base_path+'idiom_test.de',sep='\n', header=None,names=['de'])
# info = pd.read_csv(base_path+'idiom_test.info',sep='\t',header=None,names=["spread", "en_idiom", "de_idiom",'frequency'])
# data_ende = pd.concat([en,de,info],axis=1)
# data_ende.columns = ['en','de',"spread", "en_idiom", "de_idiom",'frequency']
# data_ende

In [ ]:
def clean_data(data):
  data_clean = data.copy(deep=True)
  for i in range(len(data_clean.columns)):
    data_clean.iloc[:,i] = data_clean.iloc[:,i].apply(lambda x: re.sub('&apos;', '\'', x))
    data_clean.iloc[:,i] = data_clean.iloc[:,i].apply(lambda x: re.sub('&quot;', '\"', x))
    data_clean.iloc[:,i] = data_clean.iloc[:,i].apply(lambda x: re.sub('&#124;', '|', x))
    data_clean.iloc[:,i] = data_clean.iloc[:,i].apply(lambda x: re.sub('&#93;', ']', x))
    data_clean.iloc[:,i] = data_clean.iloc[:,i].apply(lambda x: re.sub('&#91;', '[', x))
    data_clean.iloc[:,i] = data_clean.iloc[:,i].apply(lambda x: re.sub('&gt;', '>', x))
    data_clean.iloc[:,i] = data_clean.iloc[:,i].apply(lambda x: re.sub('&lt;', '<', x))

  return data_clean

In [ ]:
# Load EN-DE dataset
base_path=dir+"IdiomTranslationDS-master/en-de/"
ende = pd.read_csv(base_path+"test_compare_cleaned.csv")
data_ende = pd.DataFrame(columns=["idiom", "input"])

for i in range(len(ende)):
  if ende['Wrong'].iloc[i] != 'x':
    data_ende = data_ende.append({"idiom": ende['en_idiom'].iloc[i], "input": ende['en'].iloc[i]}, ignore_index=True)

data_ende = clean_data(data_ende)

In [ ]:
data_ende

,idiom,input
0,in this day and age,"In this day and age , when evidently many find..."
1,to be on hold,"In Romania , the election campaign will be a c..."
2,to close the door,Let us work to close the back door to illegal ...
3,to take it easy,The region is easily reached by rail ( interna...
4,to pass on,"I cannot enter into debate , but I pass on tha..."
...,...,...
1267,to pass on,I would ask you to pass on the question to the...
1268,to be in the know,"From now on , we know that the only significan..."
1269,take care !,This is the custom . It is perhaps not justifi...
1270,to beat it,"Terrorism is the child of fundamentalism , whi..."


In [ ]:
# Load paraphrase dataset
data_par = pd.read_csv(dir+'data/'+'data_cleaned.csv')
data_par_test = pd.read_csv(dir+'data/'+"data_nopref_test_split1.csv", sep="=")
data_par_test_IOB = pd.read_csv(dir+'data/'+"data_test_IOB_split1.csv", sep="=")

## Check for overlap

### Exactly equal idioms

In [ ]:
# Check for exact overlap in idioms
matches=[]
for i in range(len(data_ende)):
  for j in range(len(data_par)):
    if data_ende["idiom"].iloc[i] == data_par["Idiom"].iloc[j]:
      matches.append(data_ende["idiom"].iloc[i])

print(matches)

['in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshell', 'in a nutshel

In [ ]:
match_set = set(matches)
print(match_set)

{'in a nutshell', 'head over heels', 'a sight for sore eyes'}


#### Check for similar idioms with sequence similarity

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
a = "keep an eye on"
b = "to keep an eye on"
c = "an eye for an eye"

print(similar(a,b))
print(similar(a,c))
print(similar(b,c))


0.9032258064516129
0.5806451612903226
0.47058823529411764


In [ ]:
# Check for similar idioms
matches2=[]
for i in range(len(data_ende)):
  for j in range(len(data_par)):
    sim = similar(data_ende["idiom"].iloc[i],data_par["Idiom"].iloc[j])
    if sim>=0.7:
      matches2.append((data_ende["idiom"].iloc[i], data_par["Idiom"].iloc[j], sim))

match_list = list(set(matches2))
match_list = sorted(match_list, key= lambda x: x[2], reverse=True)
print(match_list)

[('in a nutshell', 'in a nutshell', 1.0), ('a sight for sore eyes', 'a sight for sore eyes', 1.0), ('head over heels', 'head over heels', 1.0), ('beyond a shadow of a doubt', 'beyond a shadow of doubt', 0.96), ('to kill two birds with one stone', 'kill two birds with one stone', 0.9508196721311475), ('to think outside the box', 'think outside the box', 0.9333333333333333), ('to go the extra mile', 'go the extra mile', 0.918918918918919), ('to bite the bullet', 'bite the bullet', 0.9090909090909091), ('take care !', 'take care', 0.9), ('to get the ball rolling', 'the ball rolling', 0.8205128205128205), ('to set the ball rolling', 'the ball rolling', 0.8205128205128205), ('to be miles away', 'miles away', 0.7692307692307693), ('to clear the way', 'clear the air', 0.7586206896551724), ('to close the door', 'close the books', 0.75), ('without a shadow of a doubt', 'beyond a shadow of doubt', 0.7450980392156863), ('to buy a pig in a poke', 'pig in a poke', 0.7428571428571429), ('to bite the

#### Check for similar idioms with Jaccard

In [ ]:
def jaccard_similarity(sent1, sent2): 
    
    # List the unique words in a document
    words_sent1 = set(sent1.lower().split()) 
    words_sent2 = set(sent2.lower().split())
    # Find the intersection of words
    intersection = words_sent1.intersection(words_sent2)
    # Find the union of words
    union = words_sent1.union(words_sent2)
    # Calculate Jaccard similarity 

    return float(len(intersection)) / len(union)

In [ ]:
a = "keep an eye on"
b = "to keep an eye on"
c = "an eye for an eye"

print(jaccard_similarity(a,b))
print(jaccard_similarity(a,c))
print(jaccard_similarity(b,c))


0.8
0.4
0.3333333333333333


In [ ]:
# Check for similar idioms
matches3=[]
for i in range(len(data_ende)):
  for j in range(len(data_par)):
    sim = jaccard_similarity(data_ende["idiom"].iloc[i],data_par["Idiom"].iloc[j])
    if sim>=0.6:
      matches3.append((data_ende["idiom"].iloc[i], data_par["Idiom"].iloc[j], sim))

match_list2 = list(set(matches3))
match_list2 = sorted(match_list2, key= lambda x: x[2], reverse=True)
print(match_list2)

[('head over heels', 'head over heels', 1.0), ('a sight for sore eyes', 'a sight for sore eyes', 1.0), ('in a nutshell', 'in a nutshell', 1.0), ('beyond a shadow of a doubt', 'beyond a shadow of doubt', 1.0), ('to kill two birds with one stone', 'kill two birds with one stone', 0.8571428571428571), ('to think outside the box', 'think outside the box', 0.8), ('to go the extra mile', 'go the extra mile', 0.8), ('to bite the bullet', 'bite the bullet', 0.75), ('take care !', 'take care', 0.6666666666666666), ('without a shadow of a doubt', 'beyond a shadow of doubt', 0.6666666666666666), ('to buy a pig in a poke', 'pig in a poke', 0.6666666666666666), ('to get the ball rolling', 'the ball rolling', 0.6), ('to set the ball rolling', 'the ball rolling', 0.6)]


#### Check for similar idioms with Meteor

In [ ]:
!pip install -U nltk -q
!pip install datasets -q

In [ ]:
from datasets import load_metric, load
meteor = load_metric("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# Check for similar idioms TAKES IMPOSSIBLY LONG
# matches4=[]
# for i in range(len(data_ende)):
#   for j in range(len(data_par)):
#     meteor_score= meteor.compute(predictions=[data_ende['idiom'].iloc[i]], references=[data_par["Idiom"].iloc[j]])
#     sim = meteor_score['meteor']
#     if sim>=0.6:
#       matches4.append((data_ende["idiom"].iloc[i], data_par["Idiom"].iloc[j], sim))

# match_list3 = list(set(matches4))
# match_list3 = sorted(match_list3, key= lambda x: x[2], reverse=True)
# print(match_list3)

KeyboardInterrupt: ignored

### Collect all unique english idioms from EN-DE dataset (+-132) for manual checking if they are also in the paraphrase dataset

In [ ]:
idioms_ende = set()
for i in range(len(data_ende)):
  if data_ende['idiom'].iloc[i] not in idioms_ende:
    idioms_ende.add(data_ende['idiom'].iloc[i])

print(idioms_ende)
print(len(idioms_ende))

{'to sweat blood', 'to beat it', 'in this day and age', 'to have legs', 'to go to the dogs', 'to be miles away', 'the genuine article', 'to lose face', 'to have a look around', 'written in stone', 'a square circle', 'to put up a fight', 'i hear you .', 'those were the days .', 'to pull together', 'please take your time .', 'to go places', 'to think outside the box', 'it takes two to tango .', 'take care !', 'in the heat of the moment', 'to take it easy', 'the floor is yours .', 'to raise the bar', 'you name it !', 'to go west', 'this is your chance .', 'the die is cast .', 'to rough it', 'to be spoilt for choice', 'head over heels', 'to come to the fore', 'to take the heat', 'to be in the picture', 'to close the door', 'to go the extra mile', 'without a shadow of a doubt', 'to get the ball rolling', 'enjoy your meal !', 'to be on fire', 'man and boy', 'in a nutshell', 'to buy a pig in a poke', 'to sell out', 'to get out of hand', 'have a seat .', 'short and sweet', 'to pass on', 'to sw

In [ ]:
idioms_par = set()
for i in range(len(data_par_test_IOB)):
  if data_par_test_IOB['Idiom'].iloc[i] not in idioms_par:
    idioms_par.add(data_par_test_IOB['Idiom'].iloc[i])

print(idioms_par)
print(len(idioms_par))

{'not hurt a fly or flea', 'head above water', 'deal with', 'fair-weather friend', 'chill to the marrow', 'basket case', "you can't teach an old dog new tricks", 'in the pipeline', 'in the cahoots', 'on the brink of', 'dollars to donuts', 'lean towards', 'cut down', 'cold sweat', 'cut and thrust', 'up and about', 'head and shoulders above', 'troubled waters', 'like a hole in the head', 'take care', 'black hole', 'have a soft spot', 'green with envy', 'to a T', 'head start', 'sick as a dog', 'haste makes waste', 'long run', 'a picture is worth a thousand words', 'pour cold water on', 'yellow streak', 'accidentally on purpose', 'an item', 'do the trick', 'and so on', 'play it by ear', 'out of sight, out of mind', 'take up the cudgels', 'salad days', 'lost on', 'dead as a dodo', 'pep talk', "money doesn't grow on trees", 'shed light', 'young blood', 'high up', 'playing second fiddle', 'betwixt and between', 'year-round', 'cut and dried', 'a cut above', 'larger than life', 'mind your own b

### Collect similar idioms and their sentences

In [ ]:
# Collect the idioms that were found to be similar
ende_idioms = [i[0] for i in match_list2]
ende_idioms.append("to pull together")
ende_idioms.remove("without a shadow of a doubt")

ende_idioms = list(dict.fromkeys(ende_idioms)) # remove duplicates

par_idioms = [i[1] for i in match_list2]
par_idioms.append("pull yourself together")

par_idioms = list(dict.fromkeys(par_idioms)) # remove duplicates


In [ ]:
# Collect the sentences for these idioms
ende_sentences = pd.DataFrame(columns=["idiom", "input"])
for id in ende_idioms:
  for i in range(len(data_ende)):
    if data_ende['idiom'].iloc[i] == id:
      ende_sentences = ende_sentences.append({"idiom": data_ende['idiom'].iloc[i], "input": data_ende['input'].iloc[i]}, ignore_index=True)

par_sentences = pd.DataFrame(columns=["idiom", "input"])
for id in par_idioms:
  for i in range(len(data_par_test_IOB)):
    if data_par_test_IOB['Idiom'].iloc[i] == id:
      par_sentences = par_sentences.append({"idiom": data_par_test_IOB['Idiom'].iloc[i], "input": data_par_test['input'].iloc[i]}, ignore_index=True)

In [ ]:
print(ende_sentences)
print(par_sentences)

                     idiom                                              input
0          head over heels  Sook seems to be head over heels in love with ...
1          head over heels  He first encountered the tin whistle as a yout...
2    a sight for sore eyes  In most Dalmatian towns the market is a sight ...
3    a sight for sore eyes  The Glorious DJ furniture line is a sight for ...
4            in a nutshell  In a nutshell , we are working towards a recog...
..                     ...                                                ...
159       to pull together  Its regular reports to the G-20 pull together ...
160       to pull together  The difference with this specific episode was ...
161       to pull together  We do want as part of the action plan to pull ...
162       to pull together  We must all pull together in order to maintain...
163       to pull together  Fortunately , today we know that children who ...

[164 rows x 2 columns]
                            idiom  \
0  

In [ ]:
for i in range(len(ende_sentences)):  
  print(i, ende_sentences['input'].iloc[i])
print()
for i in range(len(par_sentences)):
  print(i, par_sentences['input'].iloc[i])

0 Sook seems to be head over heels in love with her , but even though the two have some dates , Ha-kyeong rejects her new boyfriend , eventually , because the scars of her prior relationship are still too deep for her to start a new one .
1 He first encountered the tin whistle as a youth , and , before he knew it , found himself " falling head over heels in love with that magical little pipe " .
2 In most Dalmatian towns the market is a sight for sore eyes , and Zadar 's is one of the biggest and ...
3 The Glorious DJ furniture line is a sight for sore eyes in any living environment . The modern upholstered design is marked by the finest in quality and finish .
4 In a nutshell , we are working towards a recognisable and approachable European administration .
5 In a nutshell , Mr President of the Commission , what is our main aim : our main aim is neither more nor less a huge one - helping European citizens to understand that , apart from market forces , we are also interested in the ru

In [ ]:
# Remove some unsuitable sentences after manual check (not the same idiom (or not used as idiom), too long (>=3 lines))
ende_sentences = ende_sentences.drop([5,13,24,37,39,44,46,49,60,61,67,76,81,94,95,99,100,101,102,104,105,106,109,112,114,116,118,120,122,132,133,135,136,138,139,141,142,143,144,145,146,147,148,149,150,153,154,155,156,158,159,160,161,162])
ende_sentences = ende_sentences.reset_index()
ende_sentences = ende_sentences.drop("index","columns")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [ ]:
# Make comparison datasets and save to file
comparison_data = par_sentences.append(ende_sentences, ignore_index=True)
comparison_data

,idiom,input
0,head over heels,Tom and Mary are head over heels in love with ...
1,a sight for sore eyes,I can't believe that I haven't seen you in a y...
2,in a nutshell,"In a nutshell , all the new mayor was saying i..."
3,beyond a shadow of doubt,The government has clarified beyond a shadow o...
4,kill two birds with one stone,"I have to go to the bank , and on the way back..."
...,...,...
117,to pull together,"Through this vote , we are calling on the Lith..."
118,to pull together,Now I am asking the Commission and the Council...
119,to pull together,Mr Herman called on us to pull ourselves toget...
120,to pull together,"Now it is time , however , to pull ourselves t..."


In [ ]:
# Save to file
path= "/content/drive/MyDrive/MRP Idiom Translation/data/"
comparison_data.to_csv(path+"comparison_data.csv")

## Generate comparison questions

Select idiom at random, select sentence with that idiom at random (maybe probabilities biased towards how many sentences there are per dataset?), collect the three translations.

In [ ]:
# Load comparison data
path= "/content/drive/MyDrive/MRP Idiom Translation/data/Comparison sentences/"
data_base = pd.read_csv(path+"translations_comparison_baseline.csv")
data_pipe = pd.read_csv(path+"translations_comparison_pipeline.csv")
data_e2e = pd.read_csv(path+"translations_comparison_end2end.csv")


In [ ]:
# Select random idiom sentence (with equal probability for each idiom)
def rand_sents(n):
  sentences = []
  for i in range(n):
    randnr = randrange(12)
    if randnr == 0:
      sents = [0,12,13]
    if randnr == 1:
      sents = [1,14,15]
    if randnr == 2:
      sents = list(range(16,81))
      sents.append(2)
    if randnr == 3:
      sents = [3,81,82]
    if randnr == 4:
      sents = [4,83,84]
    if randnr == 5:
      sents = [5,85]
    if randnr == 6:
      sents = list(range(86,91))
      sents.append(6)
    if randnr == 7:
      sents =  [7,91]
    if randnr == 8:
      sents = list(range(92,108))
      sents.append(8)
    if randnr == 9:
      sents = list(range(108,113))
      sents.append(9)
    if randnr == 10:
      sents = [10,113,114]
    if randnr == 11:
      sents = list(range(115,122))
      sents.append(11)

    sentences.append(choice(sents))

  return sentences

In [ ]:
def print_sentences(sent_list):
  # For given indexes for sentences in the comparison data, print the idiom, 
  # english sentence and translations

  for i in sent_list:
    print("Idiom: ", data_base["Idiom"].iloc[i])
    print("English sentence: ", data_base["Input"].iloc[i])
    print("Baseline: ", data_base["Translated"].iloc[i])
    print("Pipeline: ", data_pipe["Translated"].iloc[i])
    print("End-to-end: ", data_e2e["output"].iloc[i])
    print()

In [ ]:
# Random sentence from each dataset for fixed questions
sent = [randrange(12), randrange(12,122)]
print_sentences(sent)


Idiom:  head over heels
English sentence:  Tom and Mary are head over heels in love with each other and are going to get married next month . 
Baseline:  Tom und Mary verlieben sich einander in den Kopf und werden im nächsten Monat verheiratet.
Pipeline:  Tom und Mary verlieben sich tief miteinander und werden im nächsten Monat verheiratet.
End-to-end:  Tom und Mary verlieben sich einander und werden im nächsten Monat verheiratet.

Idiom:  in a nutshell
English sentence:  In a nutshell : there were programmes being run for which there were insufficient staff resources available . 
Baseline:  Kurz gesagt: Es gab Programme, für die unzureichende Personalressourcen zur Verfügung standen.
Pipeline:  Ich sage schnell: Es gab Programme, für die unzureichende Personalressourcen zur Verfügung standen.
End-to-end:  Kurzum gesagt : Es gab Programme, für die es unzureichende Personalressourcen gab.



In [ ]:
# Random sentences from anywhere in the dataset, but equal probability for each idiom
print_sentences(rand_sents(1))

Idiom:  to get the ball rolling
English sentence:  These are all good suggestions to get the ball rolling , but they are no substitute for an overall solution . 
Baseline:  Das alles sind gute Vorschläge, um den Ball zu rollen, aber sie sind kein Ersatz für eine Gesamtlösung.
Pipeline:  Das alles sind gute Vorschläge, um anfangen zu können, aber sie ersetzen keine Gesamtlösung.
End-to-end:  Das alles sind gute Vorschläge, um den Ball in Gang zu bringen, aber sie sind kein Ersatz für eine Gesamtlösung.



In [ ]:
rand_sent.append(randrange(12))
rand_sent[1] = randrange(12,122))
print_sentences([rand_sent])

IndexError: ignored

In [ ]:
print_sentences(range(12))

Idiom:  head over heels
English sentence:  Tom and Mary are head over heels in love with each other and are going to get married next month . 
Baseline:  Tom und Mary verlieben sich einander in den Kopf und werden im nächsten Monat verheiratet.
Pipeline:  Tom und Mary verlieben sich tief miteinander und werden im nächsten Monat verheiratet.

Idiom:  a sight for sore eyes
English sentence:  I can't believe that I haven't seen you in a year ! You are a sight for sore eyes . 
Baseline:  Ich kann nicht glauben, dass ich Sie in einem Jahr nicht gesehen habe!
Pipeline:  Ich kann nicht glauben, dass ich Sie in einem Jahr nicht gesehen habe!

Idiom:  in a nutshell
English sentence:  In a nutshell , all the new mayor was saying is that the fight against crime will be taken much more seriously . 
Baseline:  Kurz gesagt, der neue Bürgermeister sagte: Der Kampf gegen die Kriminalität wird viel ernster genommen.
Pipeline:  Ich bin sicher, dass der neue Bürgermeister gesagt hat, dass die Bekämpfung 